## Import Functions and Spark Config

In [1]:
%run ./read_file.ipynb

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/24 20:07:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/24 20:07:46 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


## Set Environment

In [2]:
# env - dev / prod
# size - small / medium / large
env='prod'
size='medium'

## Data Processing

In [3]:
df_ecarbro = readCheckpoint('ecar-bro', env, size).cache()

--- 10.07209062576294 seconds ---


In [4]:
df_ecar = readCheckpoint('ecar', env, size).cache()

--- 10.93472409248352 seconds ---


In [5]:
df_ecarbro = df_ecarbro.join(df_ecar, df_ecarbro["id"] ==  df_ecar["id"], "inner").cache()
df_ecar.unpersist()
df_ecarbro.unpersist()

DataFrame[action: string, actorID: string, hostname: string, id: string, object: string, objectID: string, pid: int, ppid: int, principal: string, tid: int, timestamp: timestamp, acuity_level: string, bro_uid: string, dest_ip: string, dest_port: int, direction: string, image_path: string, l4protocol: string, src_ip: string, src_port: int, event_minute: int, event_day: int, event_hour: int, action: string, actorID: string, hostname: string, id: string, object: string, objectID: string, pid: int, ppid: int, principal: string, tid: int, timestamp: timestamp, acuity_level: string, base_address: string, command_line: string, context_info: string, data: string, dest_port: int, direction: string, end_time: timestamp, file_path: string, image_path: string, info_class: string, key: string, l4protocol: string, logon_id: string, module_path: string, new_path: string, parent_image_path: string, path: string, payload: string, privileges: string, requesting_domain: string, requesting_logon_id: strin

In [6]:
df_ecarbro

DataFrame[action: string, actorID: string, hostname: string, id: string, object: string, objectID: string, pid: int, ppid: int, principal: string, tid: int, timestamp: timestamp, acuity_level: string, bro_uid: string, dest_ip: string, dest_port: int, direction: string, image_path: string, l4protocol: string, src_ip: string, src_port: int, event_minute: int, event_day: int, event_hour: int, action: string, actorID: string, hostname: string, id: string, object: string, objectID: string, pid: int, ppid: int, principal: string, tid: int, timestamp: timestamp, acuity_level: string, base_address: string, command_line: string, context_info: string, data: string, dest_port: int, direction: string, end_time: timestamp, file_path: string, image_path: string, info_class: string, key: string, l4protocol: string, logon_id: string, module_path: string, new_path: string, parent_image_path: string, path: string, payload: string, privileges: string, requesting_domain: string, requesting_logon_id: strin

In [7]:
df_ecarbro.limit(2).show()

ERROR:root:Exception while sending command.age 24:>                (0 + 0) / 82]
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/s

Py4JError: An error occurred while calling o67.showString

In [ ]:
df_labels = readCheckpoint('labels', env, size='large')

In [ ]:
df_labels.count()

In [ ]:
df_labels.limit(5).toPandas()

In [ ]:
df_labels.toPandas().groupby('hostname')['id'].count().sort_values(ascending=False).plot(kind='bar')

In [ ]:
df_labels.toPandas().groupby('action')['id'].count().sort_values(ascending=False).plot(kind='bar')

In [ ]:
df_labels.toPandas().groupby('object')['id'].count().sort_values(ascending=False).plot(kind='bar')

In [ ]:
df_labels.distinct()